사금융기관에서 돈을 빌려주면 갚을까

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./datasets/application_train.csv')

In [ ]:
df.shape

EDA

In [ ]:
df.describe()

In [ ]:
df.columns.values

In [ ]:
df.info()

In [ ]:
%matplotlib inline
df['TARGET'].hist()

In [ ]:
df['CODE_GENDER'].value_counts().plot.pie()

In [ ]:
# 수입이 
df['AMT_INCOME_TOTAL'][df['AMT_INCOME_TOTAL']<0.4*1e6].hist()

In [ ]:
df.isna().sum().max()

In [ ]:
mean_age = df['OWN_CAR_AGE'].dropna().mean()

In [ ]:
df['OWN_CAR_AGE'] = df['OWN_CAR_AGE'].fillna(mean_age)

In [ ]:
cond = df.isna().sum() > 200000
cond.mean()

In [ ]:
df = df.dropna(axis=1, thresh=300000) #값이 x개 이상 있는 열만 남기기
df.shape

In [ ]:
df.isna().sum().sort_values().tail(10)

In [ ]:
df = df.dropna()
df.isna().sum().sum(), df.shape

In [ ]:
df.info()

In [ ]:
df.select_dtypes('float64').columns

In [ ]:
df.select_dtypes('float64').head()

In [ ]:
df.select_dtypes('float64').boxplot()

In [ ]:
df.select_dtypes('int64').head()

In [ ]:
df.select_dtypes('int64').boxplot()

In [ ]:
df.select_dtypes('object').head()

In [ ]:
df['NAME_CONTRACT_TYPE'].value_counts().plot.barh()

In [ ]:
df.groupby(['TARGET']).mean()

In [ ]:
target_group = df.groupby(['TARGET']).mean().T

In [ ]:
target_group['difference'] = abs(target_group[0]/target_group[1] - 1)

In [ ]:
target_group.sort_values(by='difference', ascending=False).head(20)

In [ ]:
df['FLAG_DOCUMENT_17'].value_counts()

In [ ]:
df.groupby(['CODE_GENDER', 'NAME_FAMILY_STATUS']).aggregate(['mean', 'max'])

In [ ]:
df.corr()

In [ ]:
df_x = df.drop(columns=['FLAG_MOBIL', 'TARGET', 'SK_ID_CURR'])
df_y = df['TARGET']
df_x.shape, df_y.shape

In [ ]:
import numpy as np
x = np.array(pd.get_dummies(df_x)).astype(np.float32)  #형변환
y = np.array(df_y).reshape([-1, 1]).astype(np.float32) #형변환, 2d변환
x.shape, y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression().fit(x, y)
model.score(x, y)

df_y 평균이 0.8인데, 점수가 0.9인 것은 성능이 굉장히 안좋은 모델

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(y, model.predict(x))

In [135]:
# precision:정밀도, recall:재현율, f1-score:정밀도와 재현율의 조화평균   support:샘플수
# 0 recall : 돈을 갚은 사람은 다 찾았지만
# 1 recall : 돈을 갚지 않은 사람은 하나도 찾지 못함
print(report)

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96    279864
         1.0       0.00      0.00      0.00     24667

    accuracy                           0.92    304531
   macro avg       0.46      0.50      0.48    304531
weighted avg       0.84      0.92      0.88    304531



In [136]:
m = 1000
#이상치가 있어서 min max 는 사용하지 못해서, 평균0 분산1로 정규화
from sklearn.preprocessing import StandardScaler
xn = x[:m]
yn = y[:m]

In [137]:
scaler = StandardScaler()
scaler.fit(xn)
xn = scaler.transform(xn)
np.min(xn), np.mean(xn), np.max(xn)

(-18.230011, 5.284692e-10, 31.60696)

In [140]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
tf.reset_default_graph()
net = slim.fully_connected(xn, 10)
hypothesis = slim.fully_connected(net, 1, activation_fn=tf.nn.sigmoid)
cost = tf.reduce_mean((yn-hypothesis)**2)
#cost = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(label=tf.cast(yn, tf.int64), logits=h))
train = tf.train.GradientDescentOptimizer(0.1).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())



In [144]:
for i in range(10000):
    sess.run(train)
    if i%1000==0:
        print(i, sess.run(cost))

0 0.25590393
1000 0.038827483
2000 0.027394667
3000 0.02082854
4000 0.017205976
5000 0.015350988
6000 0.014524655
7000 0.014042164
8000 0.013718372
9000 0.0134864375


In [146]:
# 돈 안갚은 사람 찾아냄
p = np.round(sess.run(hypothesis))
print(classification_report(yn, p))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       929
         1.0       1.00      0.83      0.91        71

    accuracy                           0.99      1000
   macro avg       0.99      0.92      0.95      1000
weighted avg       0.99      0.99      0.99      1000

